# Experimental Design
***
## Magnetism

Import the necessary packages and code to generate the magnetism results.

In [ ]:
%matplotlib inline

import numpy as np
import os

from scipy.optimize import differential_evolution

from hogben.models.magnetic import SampleYIG
from hogben.magnetism import _logl
from hogben.simulate import simulate_magnetic, reflectivity
from hogben.utils import save_plot

# Reduce size of plots for the notebook.
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

Get the fitted SLD profile and reflectivity curve for the experimentally-measured data sets of the magnetic model.

In [ ]:
sample = SampleYIG()

# Add a 0.01 uB/atom magnetic moment in the platinum layer.
sample.pt_mag.value = 0.01638

save_path = './results'
sample.sld_profile(save_path)
sample.reflectivity_profile(save_path)

Get the plot of Fisher information in the platinum layer magnetic SLD versus YIG and platinum layer thicknesses.

In [ ]:
# Number of points and counting times for each angle to simulate.
angle_times = [(0.5, 100, 20),
               (1.0, 100, 40),
               (2.0, 100, 80)]

# Range of YIG and platinum layer thicknesses to calculate over.
yig_thick_range = np.linspace(400, 900, 60)
pt_thick_range = np.concatenate((np.linspace(21.5, 30, 25), np.linspace(30, 100, 25)))

# Iterate over each YIG and platinum layer thickness being considered.
x, y, infos = [], [], []
n = len(pt_thick_range)*len(yig_thick_range) # Number of calculations.
for i, yig_thick in enumerate(yig_thick_range):
    # Display progress.
    if i % 5 == 0:
        print('>>> {0}/{1}'.format(i*len(pt_thick_range), n))

    for pt_thick in pt_thick_range:
        # Calculate the Fisher information using current thicknesses.
        g = sample.underlayer_info(angle_times, yig_thick, pt_thick)

        infos.append(g[0,0])
        x.append(yig_thick)
        y.append(pt_thick)

# Create plot of YIG and platinum layer thicknesses versus Fisher information.
fig = plt.figure(figsize=[10,8])
ax = fig.add_subplot(111, projection='3d')

# Create the surface plot and add colour bar.
surface = ax.plot_trisurf(x, y, infos, cmap='plasma')
fig.colorbar(surface, fraction=0.046, pad=0.04)

# Adjust the viewing angle of the 3D plot.
ax.view_init(elev=40, azim=140)

ax.set_xlabel('$\mathregular{YIG\ Thickness\ (\AA)}$', fontsize=11, weight='bold')
ax.set_ylabel('$\mathregular{Pt\ Thickness\ (\AA)}$', fontsize=11, weight='bold')
ax.set_zlabel('Fisher Information', fontsize=11, weight='bold')
ax.ticklabel_format(axis='z', style='sci', scilimits=(0,0))

#### Uncomment to save the plot.
# save_path = os.path.join(save_path, sample.name)
# save_plot(fig, save_path, 'underlayer_choice')

Get the plot of the log ratio of likelihoods between two models, one with an induced moment in the platinum layer and one with no moment, versus counting time for two structures: one with an optimised 26Å platinum layer thickness and the other with a sub-optimal 80Å thickness. The counting times shown are minutes per measured spin state (i.e., for this model, the total times are double the values shown here).

In [ ]:
# Define the range of times to consider (1 to 100 hours here).
times = np.linspace(40, 4000, 100)

# Number of points and split of the total counting time for each angle to simulate.
angle_splits = [(0.5, 100, 1/7),
                (1.0, 100, 2/7),
                (2.0, 100, 4/7)]

def calc_log_ratios(pt_thick):
    """Calculates log ratio of likelihoods between two models, one with an
      induced moment in the YIG sample Pt layer and one with no moment, as
      a function of measurement time.

    Args:
        pt_thick (float): Pt layer thickness to use.

    """
    ratios = []
    # Iterate over each time being considered.
    for i, total_time in enumerate(times):
        # Display progress.
        if i % 20 == 0:
            print('>>> {0}/{1}'.format(i, len(times)))
        
        temp = []
        # Get the ratio for 100 simulated data sets using the time.
        for _ in range(20):
            # Define the number of points and times for each angle.
            angle_times = [(angle, points, split*total_time)
                           for angle, points, split in angle_splits]

            # Simulate data for the YIG sample with a 1 uB/atom magnetic
            # moment in the Pt layer.
            sample = SampleYIG()
            sample.pt_mag.value = 0.01638

            structure = sample.using_conditions(pt_thick=pt_thick)
            models, datasets = simulate_magnetic(structure, angle_times, scale=1, bkg=5e-7, dq=2,
                                                 pp=True, pm=False, mp=False, mm=True)

            # Calculate the log-likelihood of a model containing the
            # Pt layer magnetic moment.
            logl_1 = _logl(models)

            # Calculate the log-likelihood of a model without the
            # Pt layer magnetic moment.
            sample.pt_mag.value = 0
            logl_2 = _logl(models)

            # Record the ratio of likelihoods.
            ratio = logl_1-logl_2
            temp.append(ratio)
            
        # Record the median ratio.
        ratios.append(np.median(temp))
    
    return ratios

# Create the plot of counting time versus log ratio of likelihoods.
fig = plt.figure(figsize=(6,7))
ax = fig.add_subplot(111)

ax.plot(1.5*times, calc_log_ratios(26), label='26Å Pt Thickness')
ax.plot(1.5*times, calc_log_ratios(80), label='80Å Pt Thickness')

ax.set_xlabel('Counting Time (min.)', fontsize=11, weight='bold')
ax.set_ylabel('Log Ratio of Likelihoods', fontsize=11, weight='bold')
ax.legend()

#### Uncomment to save the plot.
# save_path = os.path.join(save_path, 'YIG_sample')
# save_plot(fig, save_path, 'log_ratios')